In [20]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade


Requirement already up-to-date: sklearn in c:\users\lmuns\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [21]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow
from numpy.random import seed

# Read the CSV and Perform Basic Data Cleaning

In [23]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()


In [41]:
target = df["koi_disposition"]
target_names = ["CONFIRMED", "FALSE POSITIVE"]

# Select your features (columns)

In [25]:


data = df.drop("koi_disposition", axis=1)
#using the top X's from the random forest. 
feature_names = df[['koi_fpflag_co','koi_fpflag_nt', 'koi_fpflag_ss', 'koi_model_snr', 'koi_prad', 'koi_duration_err2', 'koi_duration_err1', 'koi_fpflag_ec', 'koi_prad_err2', 'koi_steff_err2', 'koi_prad_err1', 'koi_steff_err1', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_period', 'koi_depth', 'koi_duration']]
data.head()
X = data
y = target

encoded_y = pd.get_dummies(df, columns=["koi_disposition"])
encoded_y = encoded_y["koi_disposition_CONFIRMED"]
encoded_y
y = encoded_y
print(X.shape, y.shape)


(5304, 40) (5304,)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [27]:
X_train.head()



,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
5854,0,1,0,0,4.176929,0.000003,-0.000003,133.197382,0.000672,-0.000672,...,-162,4.628,0.035,-0.105,0.708,0.119,-0.051,291.05762,38.645340,15.220
3218,1,0,0,0,68.538296,0.001111,-0.001111,186.217160,0.007210,-0.007210,...,-56,2.613,0.028,-0.039,8.123,1.474,-0.295,288.03705,39.208988,13.983
2701,0,0,0,0,16.258028,0.000288,-0.000288,139.638100,0.012200,-0.012200,...,-128,4.235,0.143,-0.117,1.356,0.212,-0.259,287.62082,43.141647,13.178
1577,0,0,1,0,1.303577,0.000002,-0.000002,133.075800,0.001660,-0.001660,...,-198,4.494,0.052,-0.208,0.955,0.285,-0.102,293.62442,38.732479,14.834
627,0,0,0,0,15.573637,0.000101,-0.000101,132.442820,0.004360,-0.004360,...,-214,4.376,0.105,-0.195,1.083,0.350,-0.150,296.35867,39.115189,14.820


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [28]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled

array([[0.        , 1.        , 0.        , ..., 0.5125824 , 0.13219742,
        0.73271194],
       [1.        , 0.        , 0.        , ..., 0.37436113, 0.16822948,
        0.6229028 ],
       [0.        , 0.        , 0.        , ..., 0.35531445, 0.41963068,
        0.55144252],
       ...,
       [0.        , 1.        , 0.        , ..., 0.78773998, 0.82241916,
        0.53865957],
       [0.        , 0.        , 1.        , ..., 0.83648399, 0.30395021,
        0.58490901],
       [1.        , 0.        , 0.        , ..., 0.33148768, 0.5388321 ,
        0.72667554]])

In [29]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_train_categorical


array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

# Train the Model



In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 14,402
Trainable params: 14,402
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [32]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=0
)

In [33]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1326/1326 - 0s - loss: 0.0266 - accuracy: 0.9925
Loss: 0.026605581170787998, Accuracy: 0.9924585223197937


# Save the Model

In [40]:
model.save("models/deep_leaning.h5")